<a href="https://colab.research.google.com/github/nourbenali/Delivery/blob/main/Delivery_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import atoti as tt

In [ ]:
import pandas as p
import psycopg2
pgconn = psycopg2.connect(dbname='vente',user='postgres', password='Sassou7600',
port='5432', options='-c search_path=ventes,public')

In [ ]:
fournisseur_df = p.read_sql_query('select * from "ventes".fournisseurs',con=pgconn)
temp_df = p.read_sql_query('select * from "ventes".temps',con=pgconn)
loc_df = p.read_sql_query('select * from "ventes".localisationlivraison',con=pgconn)
fact_df = p.read_sql_query('select * from "ventes".ventesfact',con=pgconn)

print("FOURNISSEUR AFFICHAGE !!! \n" , fournisseur_df.head())

print("TEMPS AFFICHAGE !!! \n" , temp_df.head())

print("LOCALISATION AFFICHAGE !!! \n" , loc_df.head())

print("FACT AFFICHAGE !!! \n" , fact_df.head())

FOURNISSEUR AFFICHAGE !!! 
   idfournisseurs      adressefournisseurs paysfournisseurs  \
0              2  511 Maple Ave.  Apt. 1B    Iles Marshall   
1              3         662 Lyons Circle      Afghanistan   
2              6        1860 Parkside Ln.          Bhoutan   
3              7        807 Old Trail Rd.            Aruba   
4              8         618 Sheriden rd.      Antarctique   

            nomfournisseurs  
0          Bill's Dive Shop  
1  Childress Child Day Care  
2       Kermit the Pet Shop  
3     Tub's Furniture Store  
4      Toggle & Myerson Ltd  
TEMPS AFFICHAGE !!! 
   idtemps  annee  mois trimestre
0  200611   2006    11        Q4
1  200411   2004    11        Q4
2  200502   2005     2        Q1
3  200208   2002     8        Q3
4  200106   2001     6        Q2
LOCALISATION AFFICHAGE !!! 
   idpayslivraison nompayslivraison
0              99    Iles Pitcairn
1               5        Allemagne
2              28          Bhoutan
3              15            A

In [ ]:
session=tt.create_session(config={"port": 9095})
session.link()

Deleting existing "Unnamed" session to create the new one.


Open the notebook in JupyterLab with the atoti extension enabled to see this link.

In [ ]:
Vente_table = session.read_pandas(
fact_df, keys=["idfournisseurs","idpayslivraison","idtemps"], table_name="CubeVente"
)

Fsseurs_table = session.read_pandas(
fournisseur_df, keys=["idfournisseurs"], table_name="Fournisseurs"
)

Location_table = session.read_pandas(
loc_df, keys=["idpayslivraison"], table_name="PaysLivraison"
)

Time_table = session.read_pandas(
temp_df, keys=["idtemps"], table_name="TempsLivraison"
)

AtotiJavaException: A store with the name CubeVente already exists! 
If you want to recreate this store,you will have to re-create the session.

In [ ]:
Vente_table.join(Location_table, mapping={"idpayslivraison": "idpayslivraison"})
Vente_table.join(Fsseurs_table, mapping={"idfournisseurs": "idfournisseurs"})
Vente_table.join(Time_table, mapping={"idtemps": "idtemps"})

In [ ]:
Vente_table.head()

,,,quantite,montant
idtemps,idfournisseurs,idpayslivraison,,
200411,2,5,88792.0,1.370425e+09
200502,3,28,35340.0,6.310622e+08
200308,6,15,16087.0,4.814023e+08
200008,7,9,53216.0,3.477802e+09
200603,8,15,74168.0,5.779215e+09


In [ ]:
cube = session.create_cube(Vente_table)

In [ ]:
session.visualize()
cube

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

Cube(_name='CubeVente', _hierarchies=Hierarchies(), _measures=Measures())

In [ ]:
h, l, m = cube.hierarchies, cube.levels, cube.measures
h["Periode"]=[Time_table["annee"],Time_table["mois"],Time_table["trimestre"]]
cube

Cube(_name='CubeVente', _hierarchies=Hierarchies(), _measures=Measures())